In [ ]:
#importing libraries
import time
from PyPDF2 import PdfFileWriter, PdfFileReader
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import tempfile
import pandas as pd
import re
from tqdm import tqdm

#creating a function for the complete process
def pdf_processing(pdf_path):
    
    #timer for recording the time taken by each file 
    start_time = time.time()
    
    with tempfile.TemporaryDirectory() as path:
        #converting all the page of pdf to image using pdf2image library
        pages = convert_from_path(pdf_path, output_folder=path,first_page=3)
        
        #reatting the all_variable for each file
        all_text=''
        for page in tqdm(pages):

            #cropping the image in three sections
            left_crop_img=page.crop((20,105,440,2250))
            centre_crop_img=page.crop((500,105,920,2250))
            right_crop_img=page.crop((1000,105,1400,2250))
            #converting each cropped image to text
            text = str(((pytesseract.image_to_string(left_crop_img)))) + str(((pytesseract.image_to_string(centre_crop_img)))) + str(((pytesseract.image_to_string(right_crop_img))))    
            
            #cleaning the text
            text = text.replace('-\n', '')
            text = text.replace('\n', '')
            text = ''.join(e for e in text if e.isalnum() or e == ' ' or e == '/')
            text = text.replace('Sesz', 'Sex:F')
            text = text.replace('$ex', 'Sex')
            text = text.replace('FEMLE', 'FEMALE ')
            text = text.replace('8esz', 'Sex: M')
            text = text.replace('Falhers', 'Fathers')
            text = text.replace('Falhels', 'Fathers')
            text = text.replace('Famers', 'Fathers')
            text = text.replace('Fthers', 'Fathers')
            text = text.replace('Fathels', 'Fathers')
            text = text.replace('Fathefs', 'Fathers')
            text = text.replace('Fahers', 'Fathers')
            text = text.replace('Famels', 'Fathers')
            text = text.replace('Olhers', 'Mothers')
            text = text.replace('Farhers', 'Fathers')
            text = text.replace('Faﬁhels', 'Fathers')
            text = text.replace('Fanmers', 'Fathers')
            text = text.replace('Mamers', 'Mothers')
            text = text.replace("Hlsbands", "Husband")  
            text = text.replace('Faﬂiers', 'Fathers')
            text = text.replace('Fa1hers', 'Fathers')
            text = text.replace('Moﬂlers', 'Mothers')   
            text = text.replace('Farhels', 'Fathers')
            text = text.replace('Fahels', 'Fathers')
            text = text.replace('Faﬂmels', 'Fathers')
            text = text.replace('Faﬂrers', 'Fathers')   
            text = text.replace('Mdlhers', 'Mothers') 
            text = text.replace('Momers', 'Mothers') 
            text = text.replace('Eamers', 'Fathers')
            text = text.replace('Faﬂmers', 'Fathers')
            text = text.replace('Others', 'Fathers')
            text = text.replace('Me', 'Age')
            text = text.replace('A9e', 'Age')
            text = text.replace('lge', 'Age')
            text = text.replace('Ige', 'Age')
            text = text.replace('Iqe', 'Age')
            text = text.replace('Aqe', 'Age')
            text = text.replace('Pge', 'Age')
            text = text.replace('FALE', 'MALE')
            text = text.replace('Sex2MALE', 'MALE')
            text = text.replace('Sex2FEMALE', 'FEMALE')
            text = text.replace('MEMALE', 'FEMALE')
            text = text.replace('5ex', 'Sex')
            text = text.replace('6ex', 'Sex')
            text = text.replace('8ex', 'Sex')
            text = ''.join(e for e in text if e.isalnum() or e == ' ' or e == '/')
            #converting to csv format
            text = text.replace('Name', ',Name')
            text = text.replace("Father", ",Father")
            text = text.replace("Husband", ",Husband")
            text = text.replace("Mother", ",Mother")
            text = text.replace('House Number', ',House Number')
            text = text.replace('Age', ',Age')
            text = text.replace('Sex', ',Sex')
            text = text.replace('FEMALE', ',FEM-ALE\n')
            text = text.replace('MALE', ',MALE\n')
            
            #storing the text of each page to all_text
            all_text=text+all_text
    #storing all the text to a temporary file        
    f=open('temp.txt','w')
    f.write(all_text)
    f.close()
    #reading the temporary file as a variable column dataframe
    df=pd.DataFrame()
    with open('temp.txt', 'r') as f:
        for line in f:
            df = pd.concat([df, pd.DataFrame([tuple(line.strip().split(','))])], ignore_index=True)
    
    #datafram cleaning
    df.dropna(thresh=2, inplace=True)

    df1=df[df[7].isna()][df[5].apply(lambda x: 'Sex' not in str(x) and 'Age' in str(x))][[1,2,3,4,5,6]]
    df1[5]=df1[5].apply(lambda x: ''.join(re.findall(r'\d+', x)) or '00').apply(lambda x: int(x[:2]))
    df1.columns=['Name','relation','*name','House Number','Age','Sex']
    df=df[[1,2,3,4,5,7]]
    df.dropna(inplace=True)
    df[5]=df[5].apply(lambda x: ''.join(re.findall(r'\d+', x)) or '00').apply(lambda x: int(x[-2:]))
    df.columns=['Name','relation','*name','House Number','Age','Sex']
    df=pd.concat([df,df1])
    df['Sex'].apply(lambda x: str(x))
    df.dropna(inplace=True)
    df=df[df['Sex'].apply(lambda x: 'ALE' in x)]
    df.loc[df['Age']<18,'Age']=round(df[df['Age']>18]['Age'].mean())
    df['Name']= df['Name'].apply(lambda x: ' '.join(x.split()[1:]))
    df['*name'] = df['*name'].apply(lambda x: ' '.join(x.split()[1:]))
    df['House Number'] = df['House Number'].apply(lambda x: ' '.join(x.split()[2:]))
    df['House Number'] = df['House Number'].apply(lambda x: ''.join((x + 'a').split()[0].split('a')))
    df['Sex']=df['Sex'].apply(lambda x: x.replace('-',''))
    df.reset_index(drop=True, inplace=True)
    df.to_csv('final.csv', index=False, mode='a', header=False)
        
    print("--- "+str(time.time() - start_time)+" seconds ---" )
    return df

In [ ]:
temp=pd.DataFrame(columns=['Name','Relation','F-Name','House Number','Age','Sex'])
temp.to_csv('final.csv',index=False)
for i in range(1,2):
    print(i)
    df=pdf_processing('voterslist'+str(i)+'.pdf')
    